In [3]:
import torchvision.models as models
import torch
import torch.nn.functional as F
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
import cv2
import numpy as np
from sklearn.metrics import classification_report

In [4]:
def my_cv_imread(filepath):
    img = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), -1)
    return img
 
 
# 图像处理
transform = T.Compose([
    T.Resize((48, 48)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.4, 0.4, 0.4], std=[0.2, 0.2, 0.2])
])
dataset = ImageFolder(r'.\archive\raw-img', transform=transform)
 
train = int(len(dataset) * 0.8)
other_train = len(dataset) - train
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train, other_train])
 
train_dataloader_train = DataLoader(train_dataset, batch_size=1024, shuffle=True)
train_dataloader_test = DataLoader(test_dataset, batch_size=1024, shuffle=True)

In [5]:
class Net(nn.Module):
    def __init__(self, model):
        super(Net, self).__init__()
        for name, value in model.named_parameters():
            value.requires_grad = False
            
        self.vgg_layer = nn.Sequential(*list(model.children())[:-2])
        # 第一层
        self.Linear_layer1 = nn.Linear(512, 4096)
        # 第二层
        self.Linear_layer2 = nn.Linear(4096, 512)
        # 第三层
        self.Linear_layer3 = nn.Linear(512, 10)
        # drop层
        self.drop_layer = torch.nn.Dropout(p=0.5)
 
    def forward(self, x):
        x = self.vgg_layer(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = F.relu(self.Linear_layer1(x))
        x = self.drop_layer(x)
        x = F.relu(self.Linear_layer2(x))
        x = self.drop_layer(x)
        x = self.Linear_layer3(x)
        return x
 
 
vgg = models.vgg16(pretrained=True)
model = Net(vgg)

C:\Users\22973\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\22973\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
learning_rate = 0.2
max_epoch = 5
batch_size = 1024
max_batch = 16

In [7]:
optimizer = optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()
 
for epoch in range(max_epoch):
    model.train()
    batch = 0
    all_loss = 0
    trail_acc = 0
    trail_totle = 0
    allax_batch = len(train_dataloader_train)
    for train_data in train_dataloader_train:
        batch_images, batch_labels = train_data
        out = model(batch_images)
        # loss
        loss = loss_func(out, batch_labels)
        all_loss += loss
        # 预测
        prediction = torch.max(out, 1)[1]
        # 总预测准确的数量
        train_correct = (prediction == batch_labels).sum()
        # 加和数量
        trail_acc += train_correct
        # 总数量
        trail_totle += len(batch_labels)
        # 求导
        optimizer.zero_grad()
        # 反向传递
        loss.backward()
        # 向前走一步
        optimizer.step()
        batch += 1
        print("Epoch: %d/%d || batch:%d/%d average_loss: %.3f || train_acc: %.2f || loss: %.2f"
              % (epoch + 1, max_epoch, batch, allax_batch, loss, train_correct / len(batch_labels), loss))
    print("Epoch: %d/%d || acc: %d || all_loss: %.2f" % (epoch + 1, max_epoch, trail_acc / trail_totle, all_loss))

Epoch: 1/5 || batch:1/21 average_loss: 2.325 || train_acc: 0.12 || loss: 2.33
Epoch: 1/5 || batch:2/21 average_loss: 4.149 || train_acc: 0.21 || loss: 4.15
Epoch: 1/5 || batch:3/21 average_loss: 2.444 || train_acc: 0.20 || loss: 2.44
Epoch: 1/5 || batch:4/21 average_loss: 1.999 || train_acc: 0.32 || loss: 2.00
Epoch: 1/5 || batch:5/21 average_loss: 1.931 || train_acc: 0.34 || loss: 1.93
Epoch: 1/5 || batch:6/21 average_loss: 1.814 || train_acc: 0.43 || loss: 1.81
Epoch: 1/5 || batch:7/21 average_loss: 1.736 || train_acc: 0.47 || loss: 1.74
Epoch: 1/5 || batch:8/21 average_loss: 1.673 || train_acc: 0.48 || loss: 1.67
Epoch: 1/5 || batch:9/21 average_loss: 1.544 || train_acc: 0.50 || loss: 1.54
Epoch: 1/5 || batch:10/21 average_loss: 1.468 || train_acc: 0.54 || loss: 1.47
Epoch: 1/5 || batch:11/21 average_loss: 1.423 || train_acc: 0.53 || loss: 1.42
Epoch: 1/5 || batch:12/21 average_loss: 1.313 || train_acc: 0.54 || loss: 1.31
Epoch: 1/5 || batch:13/21 average_loss: 1.272 || train_acc: 0

In [8]:
transform = T.Compose([
    T.Resize((48, 48)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.4, 0.4, 0.4], std=[0.2, 0.2, 0.2])
])
 
with torch.no_grad():
    true_lable = []
    pre_lable = []
    for train_data in train_dataloader_train:
        batch_images, batch_labels = train_data
        out = model(batch_images)
        prediction = torch.max(out, 1)[1]
        true_lable += batch_labels.tolist()
        pre_lable += prediction.tolist()
 
print(classification_report(true_lable, pre_lable))

              precision    recall  f1-score   support

           0       0.72      0.80      0.76      3882
           1       0.68      0.74      0.71      2133
           2       0.77      0.76      0.77      1147
           3       0.81      0.85      0.83      1676
           4       0.81      0.84      0.82      2463
           5       0.74      0.58      0.65      1335
           6       0.74      0.56      0.63      1524
           7       0.71      0.66      0.68      1429
           8       0.87      0.91      0.89      3859
           9       0.76      0.66      0.70      1495

    accuracy                           0.77     20943
   macro avg       0.76      0.73      0.74     20943
weighted avg       0.77      0.77      0.77     20943

